In [1]:
# notebook to get numbers for, and plot for diversity trade-off

import pickle
from encode_utils.rerank_data import rerank_dist, rerank_single
from encode_utils.efficient_rerank import get_effrerank_model, run_comstyle
from encode_utils.sco_funct import weightaddprob, default_scofunct
from encode_utils.mt_scores import get_scores_auto
from transformers import AutoTokenizer
import pandas as pd
import numpy as np
import torch
import random
import os
import re
from encode_utils.new_mask_utils import randomsingle, useall
from encode_utils.eval_utils import all_lattice_multi, mean, all_unnoun_multi
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

2022-12-27 07:12:56.723629: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-27 07:12:56.723651: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
# set up information for set
col = {
    "noun_xsum": ["nounsum_reversed/", "nounxsumlargeexplodev2.csv"],
    "noun_fren": ["frtest_reversed/", "nounlargeexplodev1.csv"],
    "mt_fren": ["frtest_reversed/", "frenchlargeexplodev1.csv"],
    "mt_ende": ["detest_reversed/", "germanlargeexplodev1.csv"],
    "mt_enru": ["rutest_reversed/", "russianlargeexplodev1.csv"],
    "mt_fren_b12": ['reversed_mtfren_beam12/', 'mtfrenbeam12.csv'],
    "mt_fren_b50": ['reversed_mtfren_beam50/', 'mtfrenbeam50.csv'],
    "noun_xsum_b12": ['reversed_xsum_beam12/', 'nounxsumbeam12.csv'],
    "noun_xsum_b50": ['reversed_xsum_beam50/', 'nounxsumbeam50.csv'],
}
curcol = "noun_xsum_b12"
gsuffix = col[curcol][0]
expl_fname = col[curcol][1]
base = "outputs/graph_pickles/"+gsuffix
goldmetric = "utnoun"
explode_df = pd.read_csv("outputs/score_csvs/"+expl_fname)
#TODO switch back for french-english
if "fren" in curcol:
    SETLEN = len(os.listdir(base))
else:
    SETLEN = int(len(os.listdir(base))/2)

In [ ]:
# use noun model
if "noun" in expl_fname:
    encodemod = get_effrerank_model("noun")
# use mt model (causal)
else:
    encodemod = get_effrerank_model("comstyle")
xlm_tok = AutoTokenizer.from_pretrained("xlm-roberta-base")

In [ ]:
args = {
    'setlen':int(SETLEN),
    'tok':xlm_tok, 
    'dev':device,
    'model':encodemod,
    'explode_df':explode_df,
    'base':base,
    'goldmetric':goldmetric,
    'device':device, 
    'efficient':False,
    'noregen':False
}

In [ ]:
WEIGHT=50
#DIVWEIGHT = 1
#doadd = True
DIVWEIGHT = 1
doadd = True
    
def tokenprobdiverse (node, used):
    if hasattr(node, "prob"):
        #pcnt+=1
        tcnt = [u.token_idx for u in used].count(node.token_idx)
        return math.log(node.prob) + WEIGHT*node.score - DIVWEIGHT*tcnt
    else:
        #npcnt+=1
        return 1
    
def tokendiverse (node, used):
    if hasattr(node, "score"):
        #pcnt+=1
        tcnt = [u.token_idx for u in used].count(node.token_idx)
        return WEIGHT*node.score - DIVWEIGHT*tcnt
    else:
        #npcnt+=1
        return 1
    
def nodediverse (node, used):
    if hasattr(node, "prob"):
        #pcnt+=1
        if node in used:
            if doadd:
                return math.log(node.prob) + WEIGHT*node.score - DIVWEIGHT
        return math.log(node.prob) + WEIGHT*node.score
    else:
        #npcnt+=1
        return 1
    
def extranodediverse (node, used):
    if hasattr(node, "prob"):
        #pcnt+=1
        if node in used:
            return math.log(node.prob) + WEIGHT*node.score - DIVWEIGHT*used.count(node)
        return math.log(node.prob) + WEIGHT*node.score
    else:
        #npcnt+=1
        return 1

In [2]:
# for 100 examples, generate diverse options (up to 20), score each of them
SET = 100
i = 0
while SET>0:
    

In [ ]:
# get output for a single index out of available graphs
def test_graph_ind(ind, basedir, scofunct):
    g = pickle.load(open(basedir+str(ind), 'rb'))
    #if g['input'] in old['src']:
    #    return None, None, None
    #try:
    global usedlist
    usedlist = []
    options = []
    # TODO add a verbose option for efficient reranking so that it doesn't blow up nb
    return g['input'], g['ref'], run_pipeline(g, model, scofunct, False, 20)

# get predictions for a bunch of stuff
def get_all_preds(basedir, scofunct):
    l = len(os.listdir(basedir))
    result = []
    print("will predict total of ", l)
    for i in range(l):
        inp, r, p = test_graph_ind(i, basedir, scofunct)
        result.append({
            'src':inp,
            'hyp':p,
            'ref':r
        })
        print(i)
    result = pd.DataFrame(result)
    return result

In [ ]:
# let's use this as our evaluation metric for diversity
def get_unique_ngrams(sentence, tok, n, uns):
    toks = tok(sentence).input_ids
    #print(toks)
    for i in range(len(toks)-n):
        tmp = ""
        for j in range(i, i+n):
            tmp = tmp+"_"+str(toks[j])
        uns.add(tmp)

def cand_unique_ngrams(sentences, tok, n):
    uniques = set()
    for s in sentences:
        get_unique_ngrams(s, tok, n, uniques)
    return uniques

In [ ]:
# plots to show -> 

# Plot 1
# - include random baseline for selecting in terms of quality
# - go from left to right (number of candidates generated, and show score of each)

# Plot 2 
# - show beam search 50 baseline
# - go from left to right, show n-gram diversity metric for each

In [13]:
import pickle

In [18]:
with open("outputs/graph_pickles/exploded_mtfren_beam12/0", "rb") as file:
    res = pickle.load(file)

In [19]:
res[1]

['A slow new year for the world economy',
 'A new slow year for the world economy',
 'A slow new year for the global economy',
 'A new slow year for the global economy',
 'A Slow New Year for the World Economy',
 'A New Slow Year for the World Economy',
 'A Slow New Year for the Global Economy',
 'A New Slow Year for the Global Economy',
 'A slow new year for the world economy.',
 'A slow new year for the global economy.',
 'A new slow year for the world economy.',
 'A new slow year for the global economy.']